In [1]:
reg = Dict()

Dict{Any,Any} with 0 entries

In [30]:
ss = open("input.txt") do f
    [split(chomp(x)) for x in readlines(f)]
end

41-element Array{Array{SubString{String},1},1}:
 ["set", "i", "31"]    
 ["set", "a", "1"]     
 ["mul", "p", "17"]    
 ["jgz", "p", "p"]     
 ["mul", "a", "2"]     
 ["add", "i", "-1"]    
 ["jgz", "i", "-2"]    
 ["add", "a", "-1"]    
 ["set", "i", "127"]   
 ["set", "p", "826"]   
 ["mul", "p", "8505"]  
 ["mod", "p", "a"]     
 ["mul", "p", "129749"]
 ⋮                     
 ["add", "p", "b"]     
 ["jgz", "p", "4"]     
 ["snd", "a"]          
 ["set", "a", "b"]     
 ["jgz", "1", "3"]     
 ["snd", "b"]          
 ["set", "f", "1"]     
 ["add", "i", "-1"]    
 ["jgz", "i", "-11"]   
 ["snd", "a"]          
 ["jgz", "f", "-16"]   
 ["jgz", "a", "-19"]   

In [53]:

[s for s in ss if s[2] == "b"]

7-element Array{Array{SubString{String},1},1}:
 ["set", "b", "p"]    
 ["mod", "b", "10000"]
 ["snd", "b"]         
 ["rcv", "b"]         
 ["jgz", "b", "-1"]   
 ["rcv", "b"]         
 ["snd", "b"]         

In [62]:
reg = Dict()
i = 1
while i <= length(ss)
    global i
    s = ss[i]
#     println(i)
    offset = 1
    if s[1] == "set"
        try 
            reg[s[2]] = parse(Int,s[3])
        catch
            reg[s[2]] = get(reg,s[3],0)
        end
    elseif s[1] == "add"
        try 
            reg[s[2]] = get(reg,s[2],0) + parse(Int,s[3])
        catch
            reg[s[2]] = get(reg,s[2],0) + get(reg,s[3],0)
        end
    elseif s[1] == "mul"
        try 
            reg[s[2]] = get(reg,s[2],0) * parse(Int,s[3])
        catch
            reg[s[2]] = 0
        end
    elseif s[1] == "mod"
        try 
            reg[s[2]] = get(reg,s[2],0) % parse(Int,s[3])
        catch
            reg[s[2]] = get(reg,s[2],0) % get(reg,s[3],0)
        end
    elseif s[1] == "snd"
        try 
            reg["snd"] = push!(get(reg,"snd",[]),parse(Int,s[2])) 
        catch
            reg["snd"] = push!(get(reg,"snd",[]),get(reg,s[2],0)) 
        end
    elseif s[1] == "jgz"
        X=0
        try 
            X = parse(Int,s[2])
        catch
            X = get(reg,s[2],0)
        end
        if X > 0
            try 
                offset = parse(Int,s[3])
            catch
                offset = get(reg,s[3],0)
            end
        end
     elseif s[1] == "rcv"
        try 
            X = parse(Int,s[2])
        catch
            X = get(reg,s[2],0)
        end
        if X > 0
            println("I heard ",get(reg,"snd",["No sound"])[end])
            break
        end       
    end
    i += offset
end
    

I heard 7071


In [59]:
reg

Dict{Any,Any} with 6 entries:
  "f"   => 0
  "b"   => 7071
  "snd" => Any[8387, 9281, 551, 6152, 7244, 5947, 5254, 5397, 6928, 9552  …  70…
  "a"   => 7071
  "p"   => 0
  "i"   => 0

In [28]:
try 
    m = parse(Int,"-13")
catch
    m= "b"
end
m

-13

In [18]:
type(m)

UndefVarError: UndefVarError: type not defined

In [29]:
push!([1,23],4)

3-element Array{Int64,1}:
  1
 23
  4

## part two

In [82]:
function move(s,p,registers)
    reg = registers[p]
    offset = 1
    if s[1] == "set"
        try 
            reg[s[2]] = parse(Int,s[3])
        catch
            reg[s[2]] = get(reg,s[3],0)
        end
    elseif s[1] == "add"
        try 
            reg[s[2]] = get(reg,s[2],0) + parse(Int,s[3])
        catch
            reg[s[2]] = get(reg,s[2],0) + get(reg,s[3],0)
        end
    elseif s[1] == "mul"
        try 
            reg[s[2]] = get(reg,s[2],0) * parse(Int,s[3])
        catch
            reg[s[2]] = 0
        end
    elseif s[1] == "mod"
        try 
            reg[s[2]] = get(reg,s[2],0) % parse(Int,s[3])
        catch
            reg[s[2]] = get(reg,s[2],0) % get(reg,s[3],0)
        end
    elseif s[1] == "snd"
        try 
            reg["snd"] = push!(get(reg,"snd",[]),parse(Int,s[2])) 
        catch
            reg["snd"] = push!(get(reg,"snd",[]),get(reg,s[2],0)) 
        end
        reg["ctr"] += 1
    elseif s[1] == "jgz"
        X=0
        try 
            X = parse(Int,s[2])
        catch
            X = get(reg,s[2],0)
        end
        if X > 0
            try 
                offset = parse(Int,s[3])
            catch
                offset = get(reg,s[3],0)
            end
        end
    elseif s[1] == "rcv"
        q = registers[(p+1)%2]["snd"]
        if isempty(q)
            reg["hold"] = true
            offset = 0
        else
            reg[s[2]] = popfirst!(q)
            reg["hold"] = false
        end
    end
    return offset
end
    

move (generic function with 1 method)

In [84]:
registers = Dict(i => Dict("p" => i,"hold" => false,"snd"=>[],"ctr" => 0) for i in 0:1)
pos = [1,1]
t = 0
while !all([registers[x]["hold"] for x in 0:1])
    pos[t+1] += move(ss[pos[t+1]],t,registers)
    t = (t+1)%2
end   

In [86]:
registers[1]

Dict{String,Any} with 8 entries:
  "f"    => 0
  "ctr"  => 8001
  "b"    => 103
  "hold" => true
  "snd"  => Any[]
  "a"    => 103
  "p"    => -67
  "i"    => 0